# Leveraging Agents with Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. You can also run on a local setup, as long as you have the right IAM credentials to invoke the Claude model via Bedrock*

---

In this demo notebook, we demonstrate an implementation of Function Calling with Anthropic's Claude models via Bedrock. This notebook is inspired by the [original work](https://drive.google.com/drive/folders/1-94Fa3HxEMkxkwKppe8lp_9-IXXvsvv1) by the Anthropic Team and modified it for use with Amazon Bedrock.


This notebook need access to anthropic.claude-3-sonnet-20240229-v1:0 model in Bedrock
---

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. These use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries and can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. usuallythey are augmented by fetching information from various channels such as websites, social media platforms, and messaging apps which involve a complex workflow as shown below


### Chatbot using Amazon Bedrock

![Amazon Bedrock - Agents Interface](./images/agents.jpg)




### Building  - Key Elements

The first process in a building a contextual-aware chatbot is to identify the tools which can be called by the LLM's. 

Second process is the user request orchestration , interaction,  invoking and returing the results

### Architecture [Weather lookup]
We Search and look for the Latitude and Longitude and then invoke the weather app to get predictions

![Amazon Bedrock - Agents Interface](./images/weather.jpg)

#### Please un comment and install the libraries below if you do not have these 

In [ ]:
#!pip install langchain==0.1.17
#!pip install langchain-anthropic
#!pip install boto3==1.34.95
#!pip install faiss-cpu==1.8.0
#!pip install pypdf

#### To install the langchain-aws

you can run the `pip install langchain-aws`

to get the latest release use these commands below

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you have the required libraries and access to internet for the weather api's in this notebook. ⚠️ ⚠️ ⚠️


In [1]:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = "us-east-1",
    runtime: Optional[bool] = True,
    external_id=None,
    ep_url=None,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides"""
    target_region = region

    print(
        f"Create new client\n  Using region: {target_region}:external_id={external_id}: "
    )
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end="")
        sts = session.client("sts")
        if external_id:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
                ExternalId=external_id,
            )
        else:
            response = sts.assume_role(
                RoleArn=str(assumed_role),
                RoleSessionName="langchain-llm-1",
            )
        print(f"Using role: {assumed_role} ... sts::successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"][
            "SecretAccessKey"
        ]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name = "bedrock-runtime"
    else:
        service_name = "bedrock"

    if ep_url:
        bedrock_client = session.client(
            service_name=service_name,
            config=retry_config,
            endpoint_url=ep_url,
            **client_kwargs,
        )
    else:
        bedrock_client = session.client(
            service_name=service_name, config=retry_config, **client_kwargs
        )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [ ]:
import json
import os
import sys

import boto3
import botocore


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


bedrock_runtime = get_bedrock_client()  #
#     assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
#     region=os.environ.get("AWS_DEFAULT_REGION", None)
# )

### Anthropic Claude

#### Input

```json

"messages": [
    {"role": "user", "content": "Hello, Claude"},
    {"role": "assistant", "content": "Hello!"},
    {"role": "user", "content": "Can you describe LLMs to me?"}
        
]
{
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 100,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 0.9
} 
```

#### Output

```json
{
    'id': 'msg_01T',
    'type': 'message',
    'role': 'assistant',
    'content': [
        {
            'type': 'text',
            'text': 'Sure, the concept...'
        }
    ],
    'model': 'model_id',
    'stop_reason': 'max_tokens',
    'stop_sequence': None,
    'usage': {'input_tokens':xy, 'output_tokens': yz}}
```






### Bedrock model

Anthropic Claude

The key for this to work is to let LLM which is Claude models know about a set of `tools` that it has available i.e. functions it can call between a set of tags. This is possible because Anthropic's Claude models have been extensively trained on such tags in its training corpus.

Then present a way to call the tools in a step by step fashion till it gets the right answer. We create a set of callable functions , below e present a sample functions which can be modified to suit your needs



#### Helper function to pretty print

In [3]:
from io import StringIO
import sys
import textwrap
from langchain.llms.bedrock import Bedrock
from typing import Optional, List, Any
from langchain.callbacks.manager import CallbackManagerForLLMRun


def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

## Section 1. Connectivity and invocation

**Invoke the model to ensure connectivity** 

In [ ]:
import json

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"

messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": "What is quantum mechanics? "}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "It is a branch of physics that describes how matter and energy interact with discrete energy values ",
            }
        ],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Can you explain a bit more about discrete energies?",
            }
        ],
    },
]
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "messages": messages,
        "temperature": 0.5,
        "top_p": 0.9,
    }
)

response = bedrock_runtime.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get("body").read())
print_ww(response_body)

## Section 2 -- Tooling

###  We will connect a Vector DB and expose that as a tool 

Create a set of helper function

we will create a set of functions which we can the re use in our application
1. We will need to create a prompt template. This template helps Bedrock models understand the tools and how to invoke them.
2. Create a method to read the available tools and add it to the prompt being used to invoke Claude
3. Call function which will be responsbile to actually invoke the function with the `right` parameters
4. Format Results for helping the Model leverage the results for summarization
5. `Add to prompt`. The result which come back need to be added to the the prompt and model invoked again to get the right results

[See this notebook for more details](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/rag-solutions/rag-foundations-workshop/notebooks/05_agent_based_text_generation.ipynb)

In [4]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

### Add Tools

Recursively add the available tools

### With LangChain

Now use langchain and annotations to create the tools and invoke the functions

In [11]:
import requests

from langchain.tools import tool
from langchain.tools import StructuredTool
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain


@tool("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    url = "https://nominatim.openstreetmap.org/search"

    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    params = {"q": place, "format": "json", "limit": 1}
    response = requests.get(url, params=params, headers=headers).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None


@tool("get_weather")
def get_weather(latitude: str, longitude: str) -> dict:
    """Returns weather data for a given latitude and longitude."""
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    print_ww(f"get_weather:tool:invoked::response={response}:")
    return response.json()


# get_weather_tool = StructuredTool.from_function(get_weather)

In [12]:
tools_list = [get_lat_long, get_weather]

In [ ]:
for tools_s in tools_list:
    print_ww(
        f"Tool:name={tools_s.name}::args={tools_s.args}:: discription={tools_s.description}::"
    )

### Tooling and Agents

**Use the Default prompt template**


#### Add the retriever tooling

**Use In-Memory FAISS DB**

In [ ]:
###
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms.bedrock import Bedrock
from langchain import LLMMathChain
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
react_agent_llm = BedrockChat(
    model_id=modelId,
    client=bedrock_runtime,
    model_kwargs={"temperature": 0.1},
)

tools_list = [get_lat_long, get_weather]


react_agent = initialize_agent(
    tools_list,
    react_agent_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # SELF_ASK_WITH_SEARCH,# CONVERSATIONAL_REACT_DESCRIPTION, # ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    max_execution_time=60,
    #    handle_parsing_errors=True,
    # prompt = prompt_template
)
print_ww(
    f"The prompt:template:used:was ---- > \n\n{react_agent.agent.llm_chain.prompt}\n"
)

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings.bedrock import BedrockEmbeddings

loader = PyPDFLoader("./rag_data/Amazon_SageMaker_FAQs.pdf")
bedrock_client = get_bedrock_client()
texts = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_documents(
    loader.load()
)
embed_model = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", client=bedrock_client
)
# - create the vector store
db = FAISS.from_documents(texts, embed_model)

retriever = db.as_retriever(search_kwargs={"k": 4})
tool_search = create_retriever_tool(
    retriever=retriever,
    name="search_sagemaker_policy",
    description="Searches and returns excerpts for any question about SageMaker",
)
print_ww(tool_search.func)
tool_search.args_schema.schema()

In [ ]:
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain.memory import ConversationBufferMemory

tools_list = [get_lat_long, get_weather]


tools_list.append(tool_search)

# Construct the Tools agent and also the executor - you do not need the executor class
react_agent = initialize_agent(
    tools_list,
    react_agent_llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # SELF_ASK_WITH_SEARCH,# CONVERSATIONAL_REACT_DESCRIPTION, # ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    # handle_parsing_errors=True,
    # prompt = chat_prompt_template,
)


react_agent.invoke({"input": "can you check the weather in Marysville WA for me?"})

#### Now invoke from FAQ's

In [ ]:
react_agent.invoke({"input": "What is Amazon SageMaker Clarify?"})

## Section 2 Use the Langchain-AWS classes 
These classes having all thr latest api's and working correctly

In [ ]:
from langchain_aws.chat_models.bedrock import ChatBedrock

modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
chat = ChatBedrock(
    model_id=modelId, model_kwargs={"temperature": 0.1}, client=bedrock_runtime
)

import requests

from langchain.tools import tool
from langchain.tools import StructuredTool
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain


@tool("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    url = "https://nominatim.openstreetmap.org/search"

    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
    }
    params = {"q": place, "format": "json", "limit": 1}
    response = requests.get(url, params=params, headers=headers).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None


@tool("get_weather")
def get_weather(latitude: str, longitude: str) -> dict:
    """Returns weather data for a given latitude and longitude."""
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    print_ww(f"get_weather:tool:invoked::response={response}:")
    return response.json()


# get_weather_tool = StructuredTool.from_function(get_weather)


llm_with_tools = chat.bind_tools([get_weather, get_lat_long])
print_ww(llm_with_tools)

In [ ]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content="what is the weather like in San Francisco")]
ai_msg = llm_with_tools.invoke(messages)
ai_msg

### Use the ChatBedrock class

#### Add Retriever Tool with functions

**Add the retriever tool along with the other function calls**

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

from langchain.llms.bedrock import (
    Bedrock,
)  # required until llama_index offers direct Bedrock integration
from langchain.embeddings.bedrock import BedrockEmbeddings

from langchain_community.document_loaders import TextLoader, PyPDFLoader


loader = PyPDFLoader("./rag_data/Amazon_SageMaker_FAQs.pdf")
bedrock_client = get_bedrock_client()
texts = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_documents(
    loader.load()
)
embed_model = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", client=bedrock_client
)
# - create the vector store
db = FAISS.from_documents(texts, embed_model)

retriever = db.as_retriever(search_kwargs={"k": 4})

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool_search = create_retriever_tool(
    retriever=retriever,
    name="search_sagemaker_policy",
    description="Searches and returns excerpts for any question about SageMaker",
)
print_ww(tool_search.func)
tool_search.args_schema.schema()

In [22]:
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain.memory import ConversationBufferMemory

tools_list = [get_lat_long, get_weather]


tools_list.append(tool_search)

react_agent_llm = ChatBedrock(
    model_id=modelId,
    client=bedrock_runtime,
    # model_kwargs={"max_tokens_to_sample": 100},
    model_kwargs={"temperature": 0.1},
)

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate


prompt_template_sys = """

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [ "get_lat_long", "get_weather", "Calculator"]
Action Input: the input to the action\nObservation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}'

"""
messages = [
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=["agent_scratchpad", "input"], template=prompt_template_sys
        )
    ),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(input_variables=["input"], template="{input}")
    ),
]

chat_prompt_template = ChatPromptTemplate.from_messages(messages)
print_ww(f"from:messages:prompt:template:{chat_prompt_template}")

chat_prompt_template = ChatPromptTemplate(
    input_variables=["agent_scratchpad", "input"], messages=messages
)
print_ww(f"Crafted::prompt:template:{chat_prompt_template}")


# react_agent_llm.bind_tools = custom_bind_func

# Construct the Tools agent
react_agent = create_tool_calling_agent(
    react_agent_llm, tools_list, chat_prompt_template
)
agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools_list,
    verbose=True,
    max_iterations=5,
    return_intermediate_steps=True,
)
agent_executor.invoke({"input": "can you check the weather in Marysville WA for me?"})

In [ ]:
agent_executor.invoke({"input": "What is Amazon SageMaker Clarify?"})

## Section 3 Chat tools with history

**Use the Agent Executor chain with History for chat** 

here we are simulating as if the model had responded with the first question which was `what is clarify` and we have a follow on question

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "What kind of bias does it detect?",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
            HumanMessage(content="What is Amazon SageMaker Clarify?"),
            AIMessage(
                content="Amazon SageMaker Clarify helps detect bias in machine learning models and explain model predictions."
            ),
        ],
    }
)

#### Creating the chain of Agents + tools manually

If you want to create the Chat

In [26]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import BaseTool

from langchain.agents.format_scratchpad.tools import (
    format_to_tool_messages,
)
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser

llm_with_tools = react_agent_llm.bind_tools(tools_list)

conversational_agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
    )
    | chat_prompt_template
    | llm_with_tools
    | ToolsAgentOutputParser()
)

In [ ]:
output_t = conversational_agent.invoke(
    {"input": "What is Amazon SageMaker Clarify?", "intermediate_steps": []}
)

print_ww(output_t)

## Next steps

In this notebook we showed some basic examples of leveraging tools and agents when invoking Amazon Bedrock models using the AWS Python SDK. You're now ready to explore the other labs to dive deeper on different use-cases and patterns.